In [5]:
import cv2
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [6]:
model_path = r"C:\Users\Bima Indra\Documents\Journal PI Acne Detection\Program\best_model.h5"
model = load_model(model_path)

In [7]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [8]:
labels = ['acne-comedonica','acne-papulopustulosa','acne-conglobata']

In [9]:
saran_perawatan = {
    "acne-comedonica": "Gunakan produk non-komedogenik dan exfoliasi ringan.",
    "acne-papulopustulosa": "Gunakan obat jerawat topikal, perhatikan kebersihan wajah",
    "acne-conglobata": "Segera konsultasi ke dokter, ini tipe jerawat berat."
}

In [10]:
# Siapkan folder simpan
save_dir = r"C:\Users\Bima Indra\Documents\Hasil Capture"
os.makedirs(save_dir, exist_ok=True)

In [11]:
# Fungsi deteksi jerawat menggunakan model CNN
def detect_acne(face_roi, model, labels):
    # Resize wajah ke ukuran input model
    resized = cv2.resize(face_roi, (128, 128))
    img_array = img_to_array(resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Prediksi jenis jerawat
    pred = model.predict(img_array)
    idx = np.argmax(pred)
    label = labels[idx]
    confidence = pred[0][idx]

    return label, confidence

In [12]:
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Kamera tidak dapat dibuka.")
else:
    print("Kamera berhasil dibuka.")

Kamera berhasil dibuka.


In [26]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        wajah = frame[y:y+h, x:x+w]
        wajah = cv2.resize(wajah, (128, 128))
        img_array = img_to_array(wajah) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        pred = model.predict(img_array)
        idx = np.argmax(pred)
        label = labels[idx]
        confidence = pred[0][idx]
        saran = saran_perawatan.get(label, "Tidak ada saran.")

        # Gambar kotak di wajah
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Tampilkan label di atas kotak
        cv2.putText(frame, f"{label} ({confidence:.2f})", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Tampilkan saran di bawah layar
        cv2.putText(frame, saran, (10, frame.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("Deteksi Jerawat", frame)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break

    elif key == ord('c'):
        count = len(os.listdir(save_dir)) + 1
        filename = os.path.join(save_dir, f"capture_{count}.jpg")
        cv2.imwrite(filename, frame)
        print(f"Gambar disimpan: {filename}")

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 40ms/step


In [13]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load model multi-kelas jerawat
model = load_model("best_model.h5")  # Ganti sesuai model kamu

# Label sesuai output model (misalnya softmax 3 kelas)
label_dict = {
    0: "acne-comedonica",
    1: "acne-papulopustulosa",
    2: "acne-conglobata"
}

# Saran berdasarkan jenis jerawat
treatment_dict = {
    "acne-comedonica": "Gunakan produk non-komedogenik dan exfoliasi ringan.",
    "acne-papulopustulosa": "Gunakan obat jerawat topikal, perhatikan kebersihan wajah",
    "acne-conglobata": "Segera konsultasi ke dokter, ini tipe jerawat berat."
}

# Deteksi wajah
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Buka kamera
cap = cv2.VideoCapture(1)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_crop = frame[y:y+h, x:x+w]
        face_resized = cv2.resize(face_crop, (128, 128))
        face_input = face_resized.astype("float32") / 255.0
        face_input = np.expand_dims(face_input, axis=0)

        pred = model.predict(face_input)
        class_id = np.argmax(pred)
        confidence = np.max(pred)

        label = label_dict[class_id]
        advice = treatment_dict[label]

        # Gambar bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Tampilkan label + confidence
        cv2.putText(frame, f"{label} ({confidence:.2f})", (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Tampilkan saran pengobatan di bawah
        cv2.putText(frame, advice, (10, frame.shape[0] - 20),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Deteksi Jerawat Real-Time", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 26ms/step


In [ ]:
import cv2

for i in range(5):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Kamera dengan index {i} tersedia.")
        cap.release()
